# Used Car Price Regression Dataset - Kaggle Competition

Overview
Welcome to the 2024 Kaggle Playground Series! We plan to continue in the spirit of previous playgrounds, providing interesting an approachable datasets for our community to practice their machine learning skills, and anticipate a competition each month.

Your Goal: **The goal of this competition is to predict the price of used cars based on various attributes**.

## About the dataset

**features**
- ```id```: id of the car, not very important
- ```brand```: brand of the car
- ```model```: model of the car
- ```model_year```: year the model was made
- ```milage```: total miles on the car
- ```fuel_type```: type of fuel the car takes
- ```engine```: type of engine on the car
- ```transmission```: type of transmission on the car
- ```ext_col```: exterior color
- ```int_col```: interior color
- ```accident```: accidents the car has had in the past
- ```clean_title```: is the title clean 

**target variable**
- ```price```: price of the car


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders import TargetEncoder

In [2]:
# read in training data
data = pd.read_csv('data/raw/train.csv')
data.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


Numerical Features
- ```id```: id of the car, not very important
- ```model_year```: year the model was made
- ```milage```: total miles on the car
- ```price```: price of the car

Categorical Features
- ```brand```: brand of the car (MULTICLASS: 57 brands of cars)
- ```model```: model of the car (MULTICLASS: 1897 models of cars)
- ```fuel_type```: type of fuel the car takes (MULTICLASS: 7 types of fuels)
- ```engine```: type of engine on the car (MULTICLASS: 1117 types of engines)
- ```transmission```: type of transmission on the car (MULTICLASS: 52 types of transmissions)
- ```ext_col```: exterior color (MULTICLASS: 319 types of colors)
- ```int_col```: interior color (MULTICLASS: 156 types of colors)
- ```accident```: accidents the car has had in the past (BINARY)
- ```clean_title```: is the title clean

In [4]:
# Get the number of unique classes for each categorical feature
unique_counts = data.nunique()

# Print or view the results
print(unique_counts)

id              188533
brand               57
model             1897
model_year          34
milage            6651
fuel_type            7
engine            1117
transmission        52
ext_col            319
int_col            156
accident             2
clean_title          1
price             1569
dtype: int64


## PLAN

Handle Missing Values

- ```fuel_type```: This column has some missing values that will be dropped because this reduces the dataset by only about 2.7%

- ```accident```: This column has some missing values that will be dropped because this reduces the dataset by only about 1.3%

- ```clean_title```: It has a significant number of missing values but the whole feature will be dropped

Encode Categorical Vaues
- ```accident``` is binary already but needs to be encoded with 1's and 0's
- OneHotEncoding will be used for ```fuel_type```

Feature Engineering
- ```model_year```: Consider creating new features such as car_age (current year minus model_year) to capture how the age of the car affects the price.
- ```milage```: This numeric feature can be used as-is but consider transformations (e.g., log transformation) if the distribution is skewed.

Scaling Data

## Handling Missing Values

In [5]:
data.drop('clean_title', axis=1, inplace=True)
data.drop('id', axis=1, inplace=True)

In [6]:
data.isna().sum().value_counts

<bound method IndexOpsMixin.value_counts of brand              0
model              0
model_year         0
milage             0
fuel_type       5083
engine             0
transmission       0
ext_col            0
int_col            0
accident        2452
price              0
dtype: int64>

In [7]:
# Drop rows where 'fuel_type' or 'accident' have missing values
data.dropna(subset=['fuel_type', 'accident'], inplace=True)


In [8]:
data.isna().sum().value_counts

<bound method IndexOpsMixin.value_counts of brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
price           0
dtype: int64>

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 181067 entries, 0 to 188532
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   brand         181067 non-null  object
 1   model         181067 non-null  object
 2   model_year    181067 non-null  int64 
 3   milage        181067 non-null  int64 
 4   fuel_type     181067 non-null  object
 5   engine        181067 non-null  object
 6   transmission  181067 non-null  object
 7   ext_col       181067 non-null  object
 8   int_col       181067 non-null  object
 9   accident      181067 non-null  object
 10  price         181067 non-null  int64 
dtypes: int64(3), object(8)
memory usage: 16.6+ MB


In [ ]:
# Get the number of unique classes for each categorical feature
unique_counts = data.nunique()

# Print or view the results
print(unique_counts)

## Feature Engineering I

Extract valuable information from obtaining the 'horsepower' and 'engine_size' values for each record from the 'engine' column.

Create a binary column for whether or not the cars are luxury brands. 

Features to be engineered:

- ```horsepower```
- ```engine_size```
- ```car_age```
- ```milage_per_year```
- ```power_to_weight_ratio```

In [ ]:
data['engine']

In [ ]:
import re
# Define regex patterns to extract Horsepower (HP) and engine size (L)
hp_pattern = re.compile(r'(\d+(\.\d+)?)HP')
engine_size_pattern = re.compile(r'(\d+(\.\d+)?)L')

In [ ]:
# function to extract horsepower info from 'engine' column
def extract_horsepower(engine):
    try:
        return float(engine.split('HP')[0])
    except:
        return None

# Function to extract HP
def extract_hp(engine):
    match = hp_pattern.search(engine)
    return float(match.group(1)) if match else None



In [ ]:
data['horsepower'] = data['engine'].apply(extract_hp)

In [ ]:
data['horsepower'].isna().sum()

In [ ]:
# function to extract engine_size info from 'engine' column
def extract_engine_size(engine):
    try:
        return float(engine.split(' ')[1].replace('L', ''))
    except:
        return None

# Function to extract engine size
def extract_engine_size_L(engine):
    match = engine_size_pattern.search(engine)
    return float(match.group(1)) if match else None

In [ ]:
data['engine_size'] = data['engine'].apply(extract_engine_size_L)

In [ ]:
data['engine_size'].isna().sum()

In [ ]:
luxury_brands =  ['Mercedes-Benz', 'BMW', 'Audi', 'Porsche', 'Land', 
                    'Lexus', 'Jaguar', 'Bentley', 'Maserati', 'Lamborghini', 
                    'Rolls-Royce', 'Ferrari', 'McLaren', 'Aston', 'Maybach']
data['is_luxury_brand'] = data['brand'].apply(lambda x: 1 if x in luxury_brands else 0)

In [ ]:
data

In [ ]:
data.isna().sum().value_counts

Let us investigate any patterns behind these missing rows.

In [ ]:
missing_values_rows = data[data['horsepower'].isnull() | data['engine_size'].isnull()]
missing_values_rows

## Encode Categorical Values


In [ ]:
# List of features to apply frequency encoding
features_to_encode = ['brand', 'model', 'engine', 'transmission', 'ext_col', 'int_col']

# Frequency encoding
for feature in features_to_encode:
    # Calculate frequency of each category
    freq_encoding = data[feature].value_counts() / len(data)
    
    # Map frequencies to the original feature
    data[feature] = data[feature].map(freq_encoding)


In [10]:
### TARGET ENCODING ###
features_to_encode = ['brand', 'model', 'engine', 'transmission', 'ext_col', 'int_col']
# Initialize the target encoder
target_encoder = TargetEncoder(cols=features_to_encode, smoothing=0.3)
# Apply target encoding
target_encoded = target_encoder.fit_transform(data[features_to_encode], data['price'])

# Drop the original categorical columns
data.drop(features_to_encode, axis=1, inplace=True)

In [11]:
# Replace values in the 'accident' column
data['accident'] = data['accident'].map({
    'None reported': 0,
    'At least 1 accident or damage reported': 1
})

# Verify the transformation
print(data['accident'].value_counts())

accident
0    139724
1     41343
Name: count, dtype: int64


In [12]:
target_encoded.head()

,brand,model,engine,transmission,ext_col,int_col
0,16995.387619,12848.536585,11416.727273,30039.905922,43016.981033,27726.614211
1,30581.219992,12663.379310,12183.759494,30039.905922,31231.931601,29410.689561
2,41684.053283,23882.622951,18401.259591,30039.905922,39660.556258,27726.614211
3,53042.539432,47788.351064,41781.259459,49990.205333,41961.300670,44932.357761
4,51065.490921,64395.837209,37189.376093,46927.939397,41961.300670,29410.689561


In [13]:
numerical_features = ['brand', 'model', 'engine', 'transmission', 'ext_col', 'int_col']
scaler = StandardScaler()
array_scaled1 = scaler.fit_transform(target_encoded[numerical_features])
data_scaled1_cols = target_encoded.columns

data1_scaled = pd.DataFrame(array_scaled1, columns=data_scaled1_cols)

In [15]:
data.head()

,model_year,milage,fuel_type,accident,price
0,2007,213000,Gasoline,0,4200
1,2002,143250,Gasoline,1,4999
2,2002,136731,E85 Flex Fuel,0,13900
3,2017,19500,Gasoline,0,45000
4,2021,7388,Gasoline,0,97500


In [16]:
data2_columns = ['model_year', 'milage']
scaler = StandardScaler()
array_scaled2 = scaler.fit_transform(data[data2_columns])


data2_scaled = pd.DataFrame(array_scaled2, columns=['model_year', 'milage'])

In [ ]:
# Add the `price` column back to the scaled DataFrame for the first DataFrame
data_scaled[data.columns.difference(data_columns_to_scale)] = data[data.columns.difference(data_columns_to_scale)]

In [ ]:

fuel_type = data[['fuel_type']]

cat_encoder = OneHotEncoder(sparse_output=False)
fuel_type_1hot = cat_encoder.fit_transform(fuel_type)

# Convert the OneHotEncoded Data to a DataFrame
fuel_type_1hot_df = pd.DataFrame(fuel_type_1hot, columns=cat_encoder.get_feature_names_out(['fuel_type']))

fuel_type_1hot_df.head()

In [ ]:
def combine_dataframes(data1, data2, data3):
    # Reset indices of both DataFrames
    data1.reset_index(drop=True, inplace=True)
    data2.reset_index(drop=True, inplace=True)
    data3.reset_index(drop=True, inplace=True)

    # Concatenate the Encoded Data with the Original DataFrame
    data_processed = pd.concat([data1, data2, data3], axis=1)

    # Step 4: Drop the Original 'fuel_type' Column
    data_processed.drop('fuel_type', axis=1, inplace=True)

    return data_processed

In [ ]:
baseline_data = combine_dataframes(target_encoded, data, fuel_type_1hot_df)

In [ ]:
numerical_features = ['brand', 'model', 'engine', 'transmission', 'ext_col', 'int_col', 'model_year', 'milage', 'price']
scaler = StandardScaler()
baseline_data = scaler.fit_transform(baseline_data[numerical_features])

In [ ]:
# baseline_data.to_csv('data/processed/baseline_data.csv')

## Feature Engineering II

In [ ]:
def feature_engineering(data):
    data["car_age"] = 2024 - data["model_year"]
    data['mileage_per_year'] = data['milage'] / data['car_age']
    data['power_to_weight_ratio'] = data['horsepower'] / data['engine_size']
    return data

In [ ]:
feature_engineered_data = feature_engineering(data)
feature_engineered_data

## Scaling Numerical Features

In [ ]:
numerical_features = ['model_year', 'milage', 'car_age']
scaler = StandardScaler()
data[numerical_features] = scaler.fit_transform(data[numerical_features])

Combine One Hot Encoded Categories

In [ ]:
# Convert the Encoded Data to a DataFrame
fuel_type_1hot_df = pd.DataFrame(fuel_type_1hot, columns=cat_encoder.get_feature_names_out(['fuel_type']))

In [ ]:
data.isna().sum().value_counts

In [ ]:
# Reset indices of both DataFrames
data.reset_index(drop=True, inplace=True)
fuel_type_1hot_df.reset_index(drop=True, inplace=True)

In [ ]:
# Concatenate the Encoded Data with the Original DataFrame
data_processed = pd.concat([data, fuel_type_1hot_df], axis=1)


In [ ]:
# Step 4: Drop the Original 'fuel_type' Column
data_processed.drop('fuel_type', axis=1, inplace=True)

In [ ]:
data_processed

In [ ]:
# train_data_processed = data_processed.drop('price', axis = 1)
# train_labels_processed = data_processed['price']

In [ ]:
# train_data_processed.to_csv('../data/processed/train_data_processed.csv')
# train_labels_processed.to_csv('../data/processed/train_labels_processed.csv')